In [2]:
library(repr)
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [3]:
raw_data_url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
raw_data <- read_csv(raw_data_url,col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs","restecg", "thalach", "exang" ,"oldpeak" ,"slope" ,"ca" ,"thal" ,"num"))
tidy_data <- raw_data |>
                select(age,trestbps,chol,num) |>
                    rename("age" = "age",
                           "resting.blood.pressure" = "trestbps" ,
                           "cholestrol" = "chol",
                           "likelihood" = "num") |>
                    mutate(likelihood = case_when(likelihood == 0 ~ "Absent",
                                                  likelihood != 0 ~ "Present")) |>
                    mutate(likelihood = as.factor(likelihood))
tidy_data
                

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): ca, thal
dbl (12): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,resting.blood.pressure,cholestrol,likelihood
<dbl>,<dbl>,<dbl>,<fct>
63,145,233,Absent
67,160,286,Present
67,120,229,Present
37,130,250,Absent
41,130,204,Absent
56,120,236,Absent
62,140,268,Present
57,120,354,Absent
63,130,254,Present


training data

In [7]:
set.seed(1) ## DO NOT CHANGE

## splitting into training and testing data
data_split <- initial_split(tidy_data, prop = 0.75, strata = likelihood)
data_train <- training(data_split)
data_test <- testing(data_split)

 the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 


In [9]:
data_count <- data_train |> 
        group_by(likelihood) |>
        summarize(count = n())
data_count

likelihood,count
<fct>,<int>
Absent,123
Present,104


In [17]:
data_absent_mean <- data_train |>
        filter(likelihood == "Absent") |>
        summarize(across(age:cholestrol, mean)) |>
         rename("mean.age" = "age",
                           "mean.resting.blood.pressure" = "resting.blood.pressure" ,
                           "mean.cholestrol" = "cholestrol")

data_absent_mean

data_present_mean <- data_train |>
        filter(likelihood == "Present") |>
        summarize(across(age:cholestrol, mean)) |>
         rename("mean.age" = "age",
                           "mean.resting.blood.pressure" = "resting.blood.pressure" ,
                           "mean.cholestrol" = "cholestrol")

data_present_mean


mean.age,mean.resting.blood.pressure,mean.cholestrol
<dbl>,<dbl>,<dbl>
52.96748,130.4553,243.9106


mean.age,mean.resting.blood.pressure,mean.cholestrol
<dbl>,<dbl>,<dbl>
57.23077,135.5096,250.5673
